In [1]:
from dotenv import load_dotenv

from langchain import hub
# from langchain_community.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain.agents.tools import Tool
from langchain.agents import AgentExecutor
from langchain.agents import create_react_agent

from langchain_community.utilities import SerpAPIWrapper

load_dotenv("../.env")

True

In [2]:
# 从hub中获取React的Prompt
prompt = hub.pull("hwchase17/react")
prompt.pretty_print()

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [3]:
# 选择要使用的LLM
llm = ChatOpenAI(model='gpt-4o')
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fe2211738b0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fe221170100>, model_name='gpt-4o', openai_api_key=SecretStr('**********'), openai_proxy='')

In [4]:
# 实例化SerpAPIWrapper
search = SerpAPIWrapper()

# 准备工具列表
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="当大模型没有相关知识时，用于搜索知识",
    ),
]
tools

[Tool(name='Search', description='当大模型没有相关知识时，用于搜索知识', func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='9a385efcc26ec0f5eb7a576dedd111bb8d8a3e2b616eb78a302db84f1a435d75', aiosession=None)>)]

In [5]:
# 构建ReAct代理
agent = create_react_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': 'Search: 当大模型没有相关知识时，用于搜索知识', 'tool_names': 'Search'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')
| RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fe2211738b0>, async_client

In [6]:
# 创建代理执行器并传入代理和工具
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': 'Search: 当大模型没有相关知识时，用于搜索知识', 'tool_names': 'Search'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')
| RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.comple

In [7]:
# 调用代理执行器，传入输入数据
print("第一次运行的结果：")
agent_executor.invoke({"input": "当前Agent最新研究进展是什么?"})

第一次运行的结果：


> Entering new AgentExecutor chain...
为了回答这个问题，我需要查找最新的研究进展。

Action: Search
Action Input: "当前Agent最新研究进展 2023"['... 研究者的社会交互性和智能性问题都随着大语言模型（LLM）的发展有了新的解决方向。 为方便大家了解AI Agent领域的最新研究进展，学姐这回整理了52篇2023 ...', '一、通往AGI 的道路，AI Agent 是当前主线 · 二、AI Agent火爆出圈，实在智能行业首发塔斯Agent智能体 · 三、实在智能遥遥领先，开启AI Agent数字助理的人机 ...', '有的研究者会尝试使用大模型写小说，借鉴LSTM这个经典深度网络的思想发明RecurrentGPT，还引入了长时记忆和短时记忆机制，使模型拥有了更佳的记忆和学习 ...', '2023 年3 月起，AI Agent 领域迎来了第一次“出圈”，西部世界小镇、BabyAGI、AutoGPT 等多款重大Agent 研究项目均在短短两周内陆续上线，引发了大家对AI Agent 领域的关注。', '我们整理汇总了2023年大语言模型智能体规划技术(LLM Agent Planning)主要研究进展，并提供每个研究的概要和关键结果，希望能够启发更多相关的研究。', '嘉宾们分享了生成式AI 最新技术进展，以及其在医疗、教育、工业等重要领域落地的关键经验，探讨了AI Agent 的落地前景。 复旦NLP 实验室核心成员张奇.', '大语言模型的浪潮推动了AI Agent 相关研究快速发展，AI Agent是当前通往AGI 的主要探索路线。 ... 总的来说，自主智能体的研究进展迅速，展现出巨大的潜力和 ...', '记忆能力和长Token能力是当前很多团队的攻坚重点：刚发布的Gemeni1.5在研究中达到10M的Token长度，比GPT4整整多出两个数量级；ChatGPT也在近期官宣了“记忆能力”；国内最 ...', '通过阅读这篇报告，可以结合其他技术趋势综合了解AI Agent的最新进展和未来趋势，为研究和工作提供有益的参考。 ... 研报十：至顶智库《2023年全球生成式AI ...', '近期， 复旦大学 自然语言处理 团队（FudanNLP）推出L

{'input': '当前Agent最新研究进展是什么?',
 'output': '2023年AI Agent领域的最新研究进展包括大语言模型的发展、重大项目的上线、记忆能力和长Token能力的增强、LLM Agent Planning技术的进展，以及生成式AI在医疗、教育和工业等领域的应用。此外，复旦大学自然语言处理团队推出了一篇详尽的综述论文，涵盖了大量参考文献，全面介绍了AI Agent的历史和最新进展。'}

In [8]:
print("第二次运行的结果：")
agent_executor.invoke({"input": "当前Agent最新研究进展是什么?"})

第二次运行的结果：


> Entering new AgentExecutor chain...
要回答这个问题，我需要查找当前关于Agent最新研究进展的信息。

Action: Search
Action Input: 当前Agent最新研究进展['清华大学提出的“工具学习”框架，将AI Agents的研究推向了新的高度。在这个框架下，大模型作为“控制器”，不仅能够识别人类的意图，还能组织规划和自主纠错 ...', '... Agent研究者的社会交互性和智能性问题都随着大语言模型（LLM）的发展有了新的解决方向。为方便大家了解AI Agent领域的最新研究进展 ... 当前最先进的基于GPT ...', 'Agent一词在英文中意为“做事的主体”，在AI研究中，Agent被赋予了模仿人类主观能动性的能力，能够主动进行计划、组织、实施和学习，直至完成任务。', '通过阅读这篇报告，可以结合其他技术趋势综合了解AI Agent的最新进展和未来趋势，为研究和工作提供有益的参考。 王吉伟频道认为，对AI Agent研究和应用感 ...', '这篇综述论文针对人工智能Agent的最新进展进行了调研，重点关注了它们在增强推理、规划和工具执行能力的复杂目标的能力。', '当前位置：首页 > RPA最新资讯 > 最新资讯 > AI Agent十大研究方向，数字 ... 总的来说，自主智能体的研究进展迅速，展现出巨大的潜力和前景。', '吴恩达认为，AI智能体工作流可能比下一代基础模型带来更大的AI进展。吴恩达在演讲中重点讲述了AI智能体工作流如何迭代地进行任务，从而比传统的零样本提示 ...', '近期，Agent AI（智能体AI）技术取得了显著进展，并且其应用于实体系统中开启了以更沉浸式、动态且引人入胜的体验与代理人进行交互的新可能性。为了加快 ...', '试着从4月17号最新发布的这篇agent 综述里，提炼了6 个让agent 更智能的方法，以及2种agent 架构的适用场景。 论文链接是arxiv.org ，强烈建议每一位对agent 感兴趣的 ...', '有的研究者会尝试使用大模型写小说，借鉴LSTM这个经典深度网络的思想发明RecurrentGPT，还引入了长时记忆和短时记忆机制，使模型拥有了更佳的记忆和学习 ...']以下是当前Agent研究的最新进展：

{'input': '当前Agent最新研究进展是什么?',
 'output': '当前Agent研究的最新进展包括工具学习框架、增强推理和规划能力、AI智能体工作流、交互体验的提升，以及多种智能方法和架构的探索。这些进展展示了AI Agent在多个方面的潜力和前景。'}